In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

train_data = pd.read_csv('fashion-mnist_train.csv')
val_data = pd.read_csv('fashion-mnist_val.csv')

X_train = train_data.iloc[:, 1:].values 
y_train = train_data.iloc[:, 0].values  

X_val = val_data.iloc[:, 1:].values  
y_val = val_data.iloc[:, 0].values  

X_train = X_train / 255.0
X_val = X_val / 255.0

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [5]:
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.fc1 = nn.Linear(784, 256)  
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        return x

# 初始化模型
model = FashionMNISTModel()


In [6]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  


In [7]:
epochs = 10
for epoch in range(epochs):
    
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  
        outputs = model(images) 
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        running_loss += loss.item()
    
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  #
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {accuracy}%")


Epoch 1, Loss: 0.5510102500518163, Validation Accuracy: 85.11666666666666%
Epoch 2, Loss: 0.38987951741615934, Validation Accuracy: 85.75833333333334%
Epoch 3, Loss: 0.3481729944149653, Validation Accuracy: 87.10833333333333%
Epoch 4, Loss: 0.32167274781068167, Validation Accuracy: 87.125%
Epoch 5, Loss: 0.30105073004961014, Validation Accuracy: 87.95%
Epoch 6, Loss: 0.2864503592848778, Validation Accuracy: 88.125%
Epoch 7, Loss: 0.2725509567409754, Validation Accuracy: 88.11666666666666%
Epoch 8, Loss: 0.2581444626748562, Validation Accuracy: 88.80833333333334%
Epoch 9, Loss: 0.24809491687019666, Validation Accuracy: 89.175%
Epoch 10, Loss: 0.23594823697209358, Validation Accuracy: 88.325%


In [8]:
torch.save(model.state_dict(), 'fashion_mnist_model.pth')


In [11]:
test_data = pd.read_csv('fashion-mnist_test.csv')

X_test = test_data.iloc[:, 1:].values

X_test = X_test / 255.0

X_test = torch.tensor(X_test, dtype=torch.float32)


In [12]:
model.load_state_dict(torch.load('fashion_mnist_model.pth'))

model.eval()

with torch.no_grad(): 
    predictions = model(X_test)
    _, predicted_labels = torch.max(predictions, 1) 


In [13]:
submission = pd.DataFrame({
    'id': np.arange(len(predicted_labels)),
    'category': predicted_labels.numpy()
})

submission.to_csv('submission.csv', index=False)
